In [1]:
import math
import torch
from torch import nn
from torch import Tensor
from torch.nn  import functional as F
import gpytorch
from matplotlib import pyplot as plt
import sys
from decimal import Decimal
sys.path.append("..")
import vvkernels as vvk
import sep_vvkernels as svvk
import vvk_rbfkernel as vvk_rbf
import vvmeans as vvm
import vvlikelihood as vvll
from vfield import VField
from LBFGS import FullBatchLBFGS
import numpy as np
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
vf = VField()
f_target = vf.tgt_vec
sample_size = 100
D = vf.D
N = vf.N
def vfield_(x):
    x = x.reshape(x.shape[0],D)
    out = torch.zeros(x.shape[0], N)
    for i in range(x.shape[0]):
        out[i] = Tensor(vf(x[i])) #+ torch.randn(Tensor(vf(x[i])).size()) * 0.02
    return out
    

In [3]:
class MultitaskGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood,num_base_kernels):
        super(MultitaskGPModel, self).__init__(train_x, train_y, likelihood)
        a = torch.ones(2,2)
        chol_q = torch.tril(a)
        self.mean_module = vvm.TensorProductSubMean(gpytorch.means.ConstantMean(), num_tasks = 2)
        base_kernels = []
        for i in range(num_base_kernels):
            base_kernels.append(vvk_rbf.vvkRBFKernel())


        self.covar_module = svvk.SepTensorProductKernel(base_kernels,num_tasks = 2)

#\         self.covar_module = gpytorch.kernels.MultitaskKernel(
#             gpytorch.kernels.RBFKernel(), num_tasks=2, rank=1
#         )
       # self.covar_module = vvk.TensorProductKernel(vvk_rbf.vvkRBFKernel(), a[0,0], a[1,0], a[1,1], num_tasks = 2, rank =1,  task_covar_prior=None)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x,x)
        return gpytorch.distributions.MultitaskMultivariateNormal(mean_x, covar_x)
    
    



In [4]:
def hyper_opti(g_theta1, agg_data, training_iter,num_base_kernels):
    noises = torch.ones(agg_data.shape[0]) * 0.001
    likelihood = vvll.TensorProductLikelihood(num_tasks = 2)

    model = MultitaskGPModel(g_theta1, agg_data, likelihood,num_base_kernels)
    model.train()
    likelihood.train()
    optimizer = FullBatchLBFGS(model.parameters(), lr=.1)
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
    def closure():
        
        optimizer.zero_grad()
        output = model(g_theta1)
        #output_ll = likelihood(output)

        #loss = -likelihood.get_mll(agg_data,output_ll)
        loss = -mll(output, agg_data)
        print('Loss gp: %.3f' % ( loss))
        
        return loss
    
    loss = closure()
    loss.backward()

    for i in range(training_iter):
        options = {'closure': closure, 'current_loss': loss, 'max_ls': 10}
        loss, _, _, _, _, _, _, fail = optimizer.step(options)

        if fail:
            print('Convergence reached!')
            break
        #print(i)
    return model, likelihood

In [5]:
class design_opti_pll(nn.Module):
    def __init__(self, x):
        super(design_opti_pll, self).__init__()
        #loc = np.random.random_sample((loc_size,2))
        #self.g_theta2 = nn.Parameter(Tensor(sample))
        self.x_design = nn.Parameter(Tensor(x))
    def forward(self):
       
        #g_theta2_new = self.g_theta2 #filter_sample(self.g_theta2, 0.009)
        
        return self.x_design

In [6]:
def conduct_design_pll(x0,f_target, g_theta, agg_data, model, likelihood, training_design_iter, training_param_iter, lr_new):
    design = design_opti_pll(x0)
    loc_sample0 = loc_sample
    x_d = design.forward()
    def closure():
        optimizer.zero_grad()
        
        loss2,lower_bound, upper_bound = likelihood.get_pll(f_target,x_d, g_theta, agg_data, model, likelihood)
        #loss2 = -1. * loss2
        loss2.backward(retain_graph=True)
#         print(x_d)
#         print(lower_bound)
#         print(upper_bound)
       
        return loss2
        
        
        
    optimizer = torch.optim.LBFGS(design.parameters(), lr=lr_new, history_size=100, max_iter=100, line_search_fn="strong_wolfe")
    optimizer.step(closure)

    x_d = design.forward()
    loss2,lower_bound, upper_bound = likelihood.get_pll(f_target,x_d, g_theta, agg_data, model, likelihood)
    #loss2 = -1. * loss2
    print('Loss design: %.3f' % ( loss2))
   # print(optimizer.state_dict())
    print(x_d)
    return x_d, lower_bound, upper_bound

In [7]:
iter_hp = 75
iter_design = 40 
iter_param = 50
num_base_kernels = 3

f_target = Tensor(vf.tgt_vec) 
f_target = f_target.reshape(f_target.shape[0],1) 
tol_vector = 0.005 * torch.ones(f_target.shape)


loc_size = 260
loc_sample = np.random.random_sample((loc_size,2))
g_theta_ = (Tensor(loc_sample).clone())
agg_data1 = vfield_(g_theta_)
agg_data1 = agg_data1.flatten()


x0 = Tensor(np.array([0.5,0.7])) 
x0 = x0.reshape(1,2)
x00 = x0 
vec_x = Tensor(np.array([0.5,0.7])) 
vec_x = vec_x.reshape(1,2)

lr_new = 1.

SUCCESS = False 
FAILURE = False 
 
tol = 0.009 
print('START HYPERPARAMETERS optimization')
model, likelihood = hyper_opti(g_theta_,agg_data1,iter_hp,num_base_kernels)

print('END HYPERPARAMETERS optimization')

x0_new,lower_bound, upper_bound = conduct_design_pll(x0,f_target, g_theta_, agg_data1, model, likelihood, iter_design, iter_param, lr_new)
print(lower_bound)
print(upper_bound)
print(f_target-tol_vector)
print(f_target+tol_vector)
loc_sample = np.random.random_sample((loc_size,2))




print(x0_new)

START HYPERPARAMETERS optimization
Loss gp: 0.800
Loss gp: 0.788
Loss gp: 0.776
Loss gp: 0.751
Loss gp: 0.700
Loss gp: 0.594
Loss gp: 0.369
Loss gp: -0.122
Loss gp: -1.160
Loss gp: -2.926
Loss gp: -3.259
Loss gp: -3.264
Loss gp: -3.270
Loss gp: -3.280
Loss gp: -3.292
Loss gp: -3.303
Loss gp: -3.313
Loss gp: -3.322
Loss gp: -3.332
Loss gp: -3.340
Loss gp: -3.351
Loss gp: -3.361
Loss gp: -3.373
Loss gp: -3.381
Loss gp: -3.387
Loss gp: -3.393
Loss gp: -3.399
Loss gp: -3.404
Loss gp: -3.409
Loss gp: -3.413
Loss gp: -3.415
Loss gp: -3.419
Loss gp: -3.421
Loss gp: -3.423
Loss gp: -3.424
Loss gp: -3.426
Loss gp: -3.427
Loss gp: -3.427
Loss gp: -3.428
Loss gp: -3.429
Loss gp: -3.430
Loss gp: -3.430
Loss gp: -3.430
Loss gp: -3.431
Loss gp: -3.431
Loss gp: -3.432
Loss gp: -3.432
Loss gp: -3.433
Loss gp: -3.433
Loss gp: -3.434
Loss gp: -3.434
Loss gp: -3.435
Loss gp: -3.434
Loss gp: -3.434
Loss gp: -3.434
Loss gp: -3.434
Loss gp: -3.434
Loss gp: -3.434
Loss gp: -3.434
Loss gp: -3.434
Loss gp: -3.

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/gpytorch/utils/cholesky.py:83: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(f"A not p.d., added jitter of {jitter_new:.1e} to the diagonal", NumericalWarning)


Loss design: 0.000
Parameter containing:
tensor([[0.2012, 0.1003]], requires_grad=True)
tensor([[0.4973],
        [0.9966]], grad_fn=<CopySlices>)
tensor([[0.5025],
        [1.0035]], grad_fn=<CopySlices>)
tensor([[0.4950],
        [0.9950]])
tensor([[0.5050],
        [1.0050]])
Parameter containing:
tensor([[0.2012, 0.1003]], requires_grad=True)
